In [97]:
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_core.tools import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PDFMinerLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import create_openai_functions_agent, create_tool_calling_agent
from langchain.agents import AgentExecutor
from langchain import hub
import os


In [64]:
# 导入环境变量
x = load_dotenv()

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 2
python-dotenv could not parse statement starting at line 3


In [65]:
# 创建模型
model = ChatOpenAI(model_name="glm-z1-flash")

# 创建提示词模版
prompt = hub.pull("hwchase17/openai-functions-agent")

# 构建工具
search = TavilySearchResults(max_results=5)
print(search.run("你好"))

D:\Code\Conda\py312\envs\Mathematics_of_Deep_Learning___Using_Python\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[{'title': '你好- WordReference 汉-英词典', 'url': 'https://www.wordreference.com/zhen/%E4%BD%A0%E5%A5%BD', 'content': 'you?你好吗？(Nǐhǎo ma?). The literal translation ofHow are you?, 你怎么样？(Nǐ zěnmeyàng?), meansWhat happened to you?标题中含有单词 \'你好\' 的论坛讨论：(name), 你好！or 你好，(name)!ni hao (你好) / nin hao (您好)Pronunciation: 你好吗, 你呢你好你好 !你好 (nihao)你好 嗨 是你呀你好猛哦你好色你好辛苦好赖：你<好赖>也是大北的维修工啊访问Chinese 论坛。帮助 WordReference：在论坛上提问。Go toPreferencespage and choose from different actions for taps or mouse clicks.查看 "你好" 的谷歌机器翻译。其他语言：西班牙语|法语|葡萄牙语|意大利语|德语|荷兰语|瑞典语|波兰语|罗马尼亚语|捷克语|希腊语|土耳其语|日语|韩语|阿拉伯语链接：⚙️偏好|缩略语|隐私政策|服务条款|支持 WR|论坛|建议 | 主要翻译 | 中文 | [...] | 英语 | 你好 |  | hello, hi, how are you? How do you do |  | 有所遗漏？报告错误或提出改进建议 | 主要翻译 | 英语 |  | 中文 | Hi!interj | informal(Hello!)(非正式用语) | SCSimplified Chinese喂, 嗨, 你好 |  | Hi, John, how are you today? |  | 喂，约翰，你今天好吗? | shalominterj | (Hebrew word for peace)(希伯来语中问候语，表示“平安”的意思) | SCSimplified Chinese你好nǐ hǎoTCTraditional Chinese你好 |  | (希伯来语中道别语，表示“平安”的意思) | SCSimplified Chinese

In [75]:
# 创建分词器
embedding_model = OpenAIEmbeddings(
    model = "embedding-3" ,
    api_key = os.environ['OPENAI_API_KEY'] ,
    base_url = os.environ['OPENAI_BASE_URL']
)
embedding_model

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000020229AB5F70>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000020229AB5730>, model='embedding-3', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base='https://open.bigmodel.cn/api/paas/v4/', openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [76]:
# 获取文档 本地化
pdf_url = "https://storage.googleapis.com/deepmind-media/Era-of-Experience%20/The%20Era%20of%20Experience%20Paper.pdf"

loader = PDFMinerLoader(pdf_url)
docs = loader.load()
docs

[Document(metadata={'producer': 'pdfTeX-1.40.23', 'creator': 'TeX', 'creationdate': '2025-04-10T15:30:54+00:00', 'moddate': '2025-04-10T15:30:54+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.23 (TeX Live 2021) kpathsea version 6.3.3', 'trapped': 'False', 'total_pages': 11, 'source': 'https://storage.googleapis.com/deepmind-media/Era-of-Experience%20/The%20Era%20of%20Experience%20Paper.pdf'}, page_content='Welcome to the Era of Experience\n\nDavid Silver, Richard S. Sutton*\n\nWe stand on the threshold of a new era in artificial intelligence that promises to achieve an unprece-\ndented level of ability. A new generation of agents will acquire superhuman capabilities by learning pre-\ndominantly from experience. This note explores the key characteristics that will define this upcoming era.\n\nAbstract\n\nThe Era of Human Data\n\nArtificial intelligence (AI) has made remarkable strides over recent years by training on massive amounts of\nhuman-generated data and

In [77]:
# 创建分割器
splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', '. ', '? ', '! ', '; ', '。', '？', '！'],  # 中英文分割符
        chunk_size=100,  # 每个块的最大字符数
        chunk_overlap=50,  # 块间重叠字符数
        length_function=len  # 使用字符长度而非token计数
)
split_docs = splitter.split_documents(docs)
print(len(split_docs))
split_docs

642


[Document(metadata={'producer': 'pdfTeX-1.40.23', 'creator': 'TeX', 'creationdate': '2025-04-10T15:30:54+00:00', 'moddate': '2025-04-10T15:30:54+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.23 (TeX Live 2021) kpathsea version 6.3.3', 'trapped': 'False', 'total_pages': 11, 'source': 'https://storage.googleapis.com/deepmind-media/Era-of-Experience%20/The%20Era%20of%20Experience%20Paper.pdf'}, page_content='Welcome to the Era of Experience\n\nDavid Silver, Richard S. Sutton*'),
 Document(metadata={'producer': 'pdfTeX-1.40.23', 'creator': 'TeX', 'creationdate': '2025-04-10T15:30:54+00:00', 'moddate': '2025-04-10T15:30:54+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.23 (TeX Live 2021) kpathsea version 6.3.3', 'trapped': 'False', 'total_pages': 11, 'source': 'https://storage.googleapis.com/deepmind-media/Era-of-Experience%20/The%20Era%20of%20Experience%20Paper.pdf'}, page_content='\nWe stand on the threshold of a new era in artificial i

In [93]:
# 创建向量数据库 # Embedding-3 限制api请求数量为64 所以分批导入
db_path = "faiss_index"  # 数据库保存路径
for i in range(0, len(split_docs), 50):
    batch = split_docs[i:i+50]
    vector_db = FAISS.from_documents(batch, embedding_model)
    vector_db.save_local(db_path)
    print(f'FAISS 数据库本地化保存成功！{i}')

FAISS 数据库本地化保存成功！0
FAISS 数据库本地化保存成功！50
FAISS 数据库本地化保存成功！100
FAISS 数据库本地化保存成功！150
FAISS 数据库本地化保存成功！200
FAISS 数据库本地化保存成功！250
FAISS 数据库本地化保存成功！300
FAISS 数据库本地化保存成功！350
FAISS 数据库本地化保存成功！400
FAISS 数据库本地化保存成功！450
FAISS 数据库本地化保存成功！500
FAISS 数据库本地化保存成功！550
FAISS 数据库本地化保存成功！600


In [104]:
# 构建检索器
retriever = vector_db.as_retriever(search_kwargs={"k": 3})
retriever_tool = create_retriever_tool(
    retriever,  # 检索器
    "search_agent",  # 工具名称
    "核心观点是人工智能（AI）正从“人类数据时代”迈向“经验时代”，这将解锁超越人类能力的智能体。"  # 工具描述
)
# 创建工具列表
tools = [search, retriever_tool]

In [111]:
# 构建Agent工具
agent = create_tool_calling_agent(
    llm=model,  # 模型
    prompt=prompt,  # 提示词模版
    tools=tools  # Agent工具
)

#
agents = AgentExecutor(agent = agent ,
              tools = tools,
              verbose = True  # 输出详细信息
                )

In [113]:
msgs = agents.invoke({"input": "2025年人工智能的发展趋势是什么？"})
print(msgs['output'])



> Entering new AgentExecutor chain...
<think>
嗯，用户问的是2025年人工智能的发展趋势。我需要先确定应该调用哪个工具。提供的工具有两个：tavily_search_results_json和search_agent。tavily是用于搜索当前事件，而search_agent的核心观点是AI从数据时代转向经验时代，可能更侧重技术趋势分析。

首先，用户的问题是关于未来的趋势，属于预测性的内容，可能需要最新的信息。tavily的描述说它适合回答当前事件的问题，但不确定是否涵盖未来趋势。而search_agent可能更专注于AI发展的观点，特别是像经验时代这样的概念，可能更符合趋势分析的需求。

不过，可能需要进一步分析。比如，search_agent的描述提到“解锁超越人类能力的智能体”，这可能涉及技术突破，而tavily的搜索可能更直接获取最新的资料。考虑到用户的问题需要最新的预测，可能两者都可以，但需要选择更合适的。

或者，可能需要同时调用两个工具？但根据规则，每次只能调用一个工具。这时候可能需要看哪个更适合。假设search_agent更专注于AI发展的观点，可能更适合回答趋势问题。或者，可能tavily的搜索能提供更具体的资料，比如最近的报告或预测。

再仔细看工具描述：tavily是优化过的搜索引擎，用于当前事件，但用户的问题是关于2025年的趋势，属于未来预测，可能不在当前事件范围内。这时候可能search_agent更适合，因为它可能包含对未来的分析。或者，可能需要更仔细判断。

或者，可能应该用tavily，因为虽然问题涉及未来，但可能搜索到的资料会包含专家对未来的预测。比如，搜索“2025年人工智能发展趋势”可能会得到相关文章或报告。因此，可能应该调用tavily_search_results_json，参数是“2025年人工智能的发展趋势”。

或者，search_agent的参数是query，可能同样适用。但根据工具描述，search_agent的核心观点是AI从数据到经验时代，可能更侧重技术方向，而用户的问题更广泛。这时候可能需要优先用tavily，因为更通用。

所以最终决定调用tavily_search_results_json，参数为“2025年人工智能的发展趋势”。
</think>
